# Extended Resolution Analysis with MSSR and Manders Coefficient Calculation

This Jupyter notebook provides a framework for conducting extended resolution analysis using the MSSR (Mean Shift Super Resolution) approach on fluorescence microscopy images, followed by Manders' coefficient calculation to quantify channel colocalization. The process leverages the `napari-superres` library for enhanced spatial resolution, operating on OME-Zarr files extracted from `.oib` files. Regions of Interest (ROIs) are defined manually in ImageJ and saved for batch processing, facilitating high-throughput colocalization studies.

### Reference:
**Torres E. et al.**, "Extending resolution within a single imaging frame," *Nature Communications*, 2022. doi:10.1038/s41467-022-34693-9

**Script Author:** Adan Guerrero  
**Contact:** adan.guerrero@ibt.unam.mx

---

### Workflow Overview

1. **Resolution Extension (MSSR):**  
   - The raw fluorescence images undergo MSSR processing to achieve enhanced resolution. The MSSR technique helps resolve finer details within a single imaging frame, especially useful for high-resolution microscopy.

2. **Manders' Coefficient Calculation:**  
   - Following MSSR processing, Manders' coefficients are calculated to assess colocalization between specific fluorescence channels, facilitating an in-depth analysis of spatial relationships among cellular components.

3. **Saving Results:**  
   - Analysis results, including computed Manders' coefficients, are saved in a CSV file (i.e. `Toxin_vs_Mito_all_manders_results.csv`). This CSV file enables easy access for reloading data and further statistical analysis or validation.

---

### Experimental Design

The experimental setup focuses on analyzing the colocalization of toxins with major cellular structures through different fluorescence channels, captured on an Olympus FV1000 microscope with a 60x objective at 60 nm per pixel resolution. Specific configurations were established for different cellular structures and components, as detailed below.

- **Image Channels:**  
  The microscopy images contain four channels:
    - **C1 (Blue):** DAPI (nucleus)
    - **C2 (Cyan/Green):** Specific markers (e.g., toxin, ER)
    - **C3 (Red):** Markers like actin or other cytoskeletal elements
    - **C4 (Gray):** Additional markers (e.g., Mitochondria, Endosomes)

- **Regions of Interest (ROIs):**  
   ROIs measuring \(5 \times 5 \, \mu \text{m}^2\) were manually defined using the rectangle tool in FIJI (ImageJ v1.54f) and saved as a `.zip` file (`Analyze → Tools → ROI Manager → More → Save`). These ROIs allow focused analysis of specific areas for colocalization.

### Data Structure

The results are saved in `Toxin_vs_Mito_all_manders_results.csv` with the following columns:

- **roi**: Identifier for each ROI, detailing the specific image region analyzed.
- **m1**: Manders' coefficient for one channel, representing colocalization with a specific component (e.g., toxin).
- **m2**: Manders' coefficient for the second channel, representing colocalization with another component (e.g., mitochondria).
- **sample**: Sample identifier, useful for distinguishing different experimental conditions or replicates.

Each row provides the colocalization results for an individual ROI, allowing for statistical evaluation of colocalization across multiple samples. This structure supports batch analysis for examining experimental variations in colocalization among different cellular structures.

### Colocalization Comparisons (described on the `data_configurations` dictionary):

1. **Actin Channels:**
   - **C1 + C2:** Nucleus (DAPI) and Toxin
   - **C2 + C3:** Toxin and actin, colocalization
   - **Additional Channels:** Similar analysis for lysosomal and mitochondrial markers

2. **Endoplasmic Reticulum (ER) Channels:**
   - **C1 + C3:** Nucleus and Toxin
   - **C2 + C3:** ER and toxin colocalization

3. **Endosomal Channels:**
   - **C1 + C2:** Nucleus and Toxin
   - **C2 + C4:** Toxin and endosomes, colocalization

Additional colocalization analyses focus on cytoplasmic, extracellular and nuclear regions, with vesicular markers (ER, Endosomes, Lysosomes, and Mitochondria) studied in their colocalization with toxins, enabling insights into cellular dynamics and toxin distribution.

--- 


## Install Libraries

In [1]:
print("Installing necessary libraries...")
!pip install os > /dev/null 2>&1
!pip install ome-zarr > /dev/null 2>&1
!pip install PyQt5 > /dev/null 2>&1
!pip install napari[all] > /dev/null 2>&1
!pip install natsort  > /dev/null 2>&1
!pip install git+https://github.com/RoccoDAnt/napari-superres.git > /dev/null 2>&1
print("Libraries installed successfully.")

Installing necessary libraries...
Libraries installed successfully.


## Data dictionary

In [2]:
import os
wd = '/home/jovyan/LNMA/bravoa/data/'

data_configurations = {
        "Toxin_vs_actin": {
        "directory_path": os.path.join(wd, "New2 Fig para colocalizacion Manders-actina"),
        "order_types": ["Cit"],
        "ch_m1": 1,
        "ch_m2": 2,
        "df_name": "Toxin_vs_actin"
    },
    "Toxin_vs_Nucleus_a": {
        "directory_path": os.path.join(wd, "New2 Fig para colocalizacion Manders-actina"),
        "order_types": ["Nuc"],
        "ch_m1": 1,
        "ch_m2": 0,
        "df_name": "Toxin_vs_Nucleus_a"
    },
    "Toxin_vs_actin_ord": {
        "directory_path": os.path.join(wd, "New2 Fig para colocalizacion Manders-actina"),
        "order_types": ["-ord"],
        "ch_m1": 1,
        "ch_m2": 2,
        "df_name": "Toxin_vs_actin_ord"
    },
    "Toxin_vs_actin_desord": {
        "directory_path": os.path.join(wd, "New2 Fig para colocalizacion Manders-actina"),
        "order_types": ["esord"],
        "ch_m1": 1,
        "ch_m2": 2,
        "df_name": "Toxin_vs_actin_desord"
    },
    "Toxin_vs_actin_mutord": {
        "directory_path": os.path.join(wd, "New2 Fig para colocalizacion Manders-actina"),
        "order_types": ["Mutord"],
        "ch_m1": 1,
        "ch_m2": 2,
        "df_name": "Toxin_vs_actin_mutord"
    },
    "Toxin_vs_Nucleus_ER": {
        "directory_path": os.path.join(wd, "New2 Fig para colocalizacion Manders-ER"),
        "order_types": ["Nuc"],
        "ch_m1": 2,
        "ch_m2": 0,
        "df_name": "Toxin_vs_Nucleus_ER"
    },
    "Toxin_vs_ER": {
        "directory_path": os.path.join(wd, "New2 Fig para colocalizacion Manders-ER"),
        "order_types": ["Cit"],
        "ch_m1": 2,
        "ch_m2": 1,
        "df_name": "Toxin_vs_ER"
    },
    "Toxin_vs_Nucleus_L": {
        "directory_path": os.path.join(wd, "New2 Fig para colocalizacion Manders-Lizo"),
        "order_types": ["Nuc"],
        "ch_m1": 1,
        "ch_m2": 0,
        "df_name": "Toxin_vs_Nucleus_L"
    },
    "Toxin_vs_Lizo": {
        "directory_path": os.path.join(wd, "New2 Fig para colocalizacion Manders-Lizo"),
        "order_types": ["Cit"],
        "ch_m1": 1,
        "ch_m2": 2,
        "df_name": "Toxin_vs_Lizo"
    },
    "Toxin_vs_Nucleus_M": {
        "directory_path": os.path.join(wd, "New2 Fig para colocalizacion Manders-Mito"),
        "order_types": ["Nuc"],
        "ch_m1": 1,
        "ch_m2": 0,
        "df_name": "Toxin_vs_Nucleus_M"
    },
    "Toxin_vs_Mito": {
        "directory_path": os.path.join(wd, "New2 Fig para colocalizacion Manders-Mito"),
        "order_types": ["Cit"],
        "ch_m1": 1,
        "ch_m2": 2,
        "df_name": "Toxin_vs_Mito"
    },
    "Toxin_vs_Nucleus_E": {
        "directory_path": os.path.join(wd, "New2 Fig para colocalizacion Manders-Endosomas"),
        "order_types": ["Nuc"],
        "ch_m1": 1,
        "ch_m2": 0,
        "df_name": "Toxin_vs_Nucleus_E"
    },
    "Toxin_vs_Endo": {
        "directory_path": os.path.join(wd, "New2 Fig para colocalizacion Manders-Endosomas"),
        "order_types": ["Cit"],
        "ch_m1": 1,
        "ch_m2": 3,
        "df_name": "Toxin_vs_Endo"
    },
    "Toxin_vs_EndoEarly": {
        "directory_path": os.path.join(wd, "Seleccion ROIs para Fig endoscitosis"),
        "order_types": ["Endo"],
        "ch_m1": 1,
        "ch_m2": 3,
        "df_name": "Toxin_vs_EndoEarly"
    },
    "Toxin_vs_mutEndoEarly": {
        "directory_path": os.path.join(wd, "Seleccion ROIs para Fig endoscitosis"),
        "order_types": ["MutEndo"],
        "ch_m1": 1,
        "ch_m2": 3,
        "df_name": "Toxin_vs_mutEndoEarly"
    },          
    "Vesicles_Toxin_vs_actin": {
        "directory_path": os.path.join(wd, "Seleccion ROIs circulares actina"),
        "order_types": ["actin"],
        "ch_m1": 1,
        "ch_m2": 2,
        "df_name": "Vesicles_Toxin_vs_actin"
    },
    "Vesicles_Toxin_vs_ER": {
        "directory_path": os.path.join(wd, "Seleccion ROIs circulares RE"),
        "order_types": ["VExt"],
        "ch_m1": 2,
        "ch_m2": 1,
        "df_name": "Vesicles_Toxin_vs_ER"
    },
    "Vesicles_Toxin_vs_Lizo": {
        "directory_path": os.path.join(wd, "Seleccion ROIs circulares Lizo"),
        "order_types": ["Lizo"],
        "ch_m1": 1,
        "ch_m2": 2,
        "df_name": "Vesicles_Toxin_vs_Lizo"
    },
    "Vesicles_Toxin_vs_Mito": {
        "directory_path": os.path.join(wd, "Seleccion ROIs circulares Mito"),
        "order_types": ["mito"],
        "ch_m1": 1,
        "ch_m2": 2,
        "df_name": "Vesicles_Toxin_vs_Mito"
    },
    "Vesicles_Toxin_vs_Endo": {
        "directory_path": os.path.join(wd, "Seleccion ROIs circulares Endosomas"),
        "order_types": ["endo"],
        "ch_m1": 1,
        "ch_m2": 3,
        "df_name": "Vesicles_Toxin_vs_Endo"
    }
}          

## Parameters

In [3]:
### Parameters for MSSR

fwhm = 8  # Full Width at Half Maximum
amp = 1  # Amplitude
order = 1  # Order
mesh = True  # Optional, default is True
ftI = False  # Optional, default is False
intNorm = False  # Optional, default is True
sigma = 2 # For the Gassian filter


## Import Libraries

In [4]:
import zarr
import pandas as pd
from natsort import natsorted
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import gaussian
from napari_superres.core_mssr import mssr_class
mssr_instance = mssr_class()

/opt/conda/lib/python3.11/site-packages/pytools/persistent_dict.py:52: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


This exception is what's causing cl equals None: clGetPlatformIDs failed: PLATFORM_NOT_FOUND_KHR
Cupy implementation is not available. Make sure you have the right version of Cupy and CUDA installed.
Optional dependecy Dask_image is not installed. Implementations using it will be ignored.


/opt/conda/lib/python3.11/site-packages/liquid_engine/__opencl__.py:125: UserWarning: tap... tap... tap... COMPUTER SAYS NO (OpenCL)!
  warnings.warn("tap... tap... tap... COMPUTER SAYS NO (OpenCL)!")


## Define functions

In [5]:
# Extract the rectangular region from the specified channel
def extract_rect_roi(image, left, top, width, height):
    return image[top:top+height, left:left+width]

# Extract the oval region from the image, covering the full image dimensions
def extract_oval_roi_from_image(image):
    # Get image dimensions
    height, width = image.shape
    
    # Calculate the center of the oval
    cx, cy = width // 2, height // 2
    
    # Generate a coordinate grid based on the image dimensions
    y, x = np.ogrid[:height, :width]
    
    # Create the oval mask centered within the full image
    mask = ((x - cx) ** 2) / (width / 2) ** 2 + ((y - cy) ** 2) / (height / 2) ** 2 <= 1
    
    # Apply the mask to the image
    oval_roi = np.where(mask, image, 0)
    
    return oval_roi

def manders_colocalization(image1, image2):
    overlap = np.logical_and(image1 > 0, image2 > 0)
    m1 = np.sum(overlap) / np.sum(image1 > 0) if np.sum(image1 > 0) > 0 else 0
    m2 = np.sum(overlap) / np.sum(image2 > 0) if np.sum(image2 > 0) > 0 else 0
    return m1, m2
    
def normalize_channel(channel, lower_percentile, upper_percentile):
    lower_bound = np.percentile(channel, lower_percentile)
    upper_bound = np.percentile(channel, upper_percentile)
    channel = np.clip(channel, lower_bound, upper_bound)
    # Avoid division by zero
    if upper_bound == lower_bound:
        return np.zeros_like(channel)
    return (channel - lower_bound) / (upper_bound - lower_bound)

def process_files_in_directory(directory_path, order_types, ch_m1, ch_m2, mssr_instance, sigma, fwhm, amp, order, mesh, ftI, intNorm):
    M = []
    for file_name in natsorted(os.listdir(directory_path)):
        if file_name.endswith('.zarr'):
            ome_zarr_path = os.path.join(directory_path, file_name)
            
            # Load the OME-Zarr file
            images = zarr.open(ome_zarr_path, mode="r")
            channel_m1 = images["image_data"][0, ch_m1, 0, :, :].astype(np.float32)
            channel_m2 = images["image_data"][0, ch_m2, 0, :, :].astype(np.float32)

            # Extract ROI metadata from the OME-Zarr file
            roi_metadata = images.attrs.get("roi_metadata", {})
            roi_metadata = [item for item in roi_metadata if any(order_type in item['name'] for order_type in order_types)]
            
            for roi_info in roi_metadata:
                left = max(roi_info['left'], 0)
                top = max(roi_info['top'], 0)
                width = roi_info['width']
                height = roi_info['height']

                # Extract ROIs
                roi_channel_m1 = extract_rect_roi(channel_m1, left, top, width, height)
                roi_channel_m2 = extract_rect_roi(channel_m2, left, top, width, height)
        
                # Apply Gaussian filter
                roi_channel_m1 = gaussian(roi_channel_m1, sigma)
                roi_channel_m2 = gaussian(roi_channel_m2, sigma)

                # Apply MSSR
                roi_channel_m1_mssr = mssr_instance.sfMSSR(img=roi_channel_m1, fwhm=fwhm, amp=amp, order=order, mesh=mesh, ftI=ftI, intNorm=intNorm)
                roi_channel_m2_mssr = mssr_instance.sfMSSR(img=roi_channel_m2, fwhm=fwhm, amp=amp, order=order, mesh=mesh, ftI=ftI, intNorm=intNorm)
                
                if roi_info['type'] == 'oval':
                    roi_channel_m1_mssr = extract_oval_roi_from_image(roi_channel_m1_mssr)
                    roi_channel_m2_mssr = extract_oval_roi_from_image(roi_channel_m2_mssr)

                # Compute Manders' colocalization coefficients after MSSR
                m1, m2 = manders_colocalization(roi_channel_m1_mssr, roi_channel_m2_mssr)

                # Extract the sample number from the file name
                sample_number_str = file_name.split('_')[0]
                sample_number = int(sample_number_str)

                # Append the results as a tuple
                M.append((roi_info['name'], round(m1, 3), round(m2, 3), sample_number))
    return M

## Loop through each configuration in the dictionary and process all files accordingly

In [6]:
for config_name, config_data in data_configurations.items():
    print(f"Processing configuration: {config_name}")
    
    # Extract parameters for each configuration
    directory_path = config_data["directory_path"]
    order_types = config_data["order_types"]
    ch_m1 = config_data["ch_m1"]
    ch_m2 = config_data["ch_m2"]
    df_name = config_data["df_name"]


    # Print out the details
    print("  Directory Path:", directory_path)
    print("  Order Types:", order_types)
    print("  Channel M1:", ch_m1)
    print("  Channel M2:", ch_m2)
    print("  DataFrame Name:", df_name)
    print("-" * 40)  # Divider between configurations
    
    # Call the function to process files in the directory with the specified parameters
    M = process_files_in_directory(
        directory_path=directory_path,
        order_types=order_types,
        ch_m1=ch_m1,
        ch_m2=ch_m2,
        mssr_instance=mssr_instance,  # Assuming these variables are defined in your environment
        sigma=sigma,
        fwhm=fwhm,
        amp=amp,
        order=order,
        mesh=mesh,
        ftI=ftI,
        intNorm=intNorm
    )

    # Specify headers when creating the DataFrame
    df = pd.DataFrame(M, columns=['roi', 'm1', 'm2', 'sample'])
    # Save the DataFrame
    df_file_name = f"{df_name}_all_manders_results.csv"
    df_file_path = os.path.join(directory_path, df_file_name)
    print(df_file_path)
    df.to_csv(df_file_path, index=False)  
    
    # Process the results in M as needed
    print("Results:", df)
    print("-" * 40)  # Divider between configurations

Processing configuration: Toxin_vs_actin
  Directory Path: /home/jovyan/LNMA/bravoa/data/New2 Fig para colocalizacion Manders-actina
  Order Types: ['Cit']
  Channel M1: 1
  Channel M2: 2
  DataFrame Name: Toxin_vs_actin
----------------------------------------


progress: 100%|██████████| 9/9 [00:00<00:00, 2699.62it/s]


/home/jovyan/LNMA/bravoa/data/New2 Fig para colocalizacion Manders-actina/Toxin_vs_actin_all_manders_results.csv
Results:                          roi     m1     m2  sample
0    0002-0680-0106-CitVerde  0.525  0.534       1
1    0002-0217-0354-CitVerde  0.487  0.496       1
2    0002-0172-0424-CitVerde  0.469  0.470       1
3    0002-0612-0114-CitVerde  0.525  0.541       1
4    0002-0564-0146-CitVerde  0.529  0.551       1
5    0002-0150-0362-CitVerde  0.498  0.528       1
6    0002-0532-0074-CitVerde  0.487  0.498       2
7    0002-0602-0044-CitVerde  0.543  0.534       2
8    0002-0616-0158-CitVerde  0.476  0.507       2
9    0002-0406-0084-CitVerde  0.510  0.527       2
10   0003-0656-0544-CitVerde  0.530  0.527      11
11   0003-0782-0744-CitVerde  0.526  0.534      11
12   0003-0862-0580-CitVerde  0.471  0.441      11
13   0003-0572-0602-CitVerde  0.512  0.504      11
14   0003-0526-0768-CitVerde  0.496  0.507      11
15   0003-0528-0484-CitVerde  0.478  0.495      12
16   0002-0

progress: 100%|██████████| 9/9 [00:00<00:00, 2585.88it/s]


/home/jovyan/LNMA/bravoa/data/New2 Fig para colocalizacion Manders-actina/Toxin_vs_Nucleus_a_all_manders_results.csv
Results:                            roi     m1     m2  sample
0   0001-0486-0400-NucleoVerde  0.434  0.421      12
1   0001-0482-0350-NucleoVerde  0.482  0.491      12
2   0001-0536-0370-NucleoVerde  0.533  0.522      12
3   0001-0492-0308-NucleoVerde  0.493  0.448      12
4   0001-0808-0452-NucleoVerde  0.469  0.446      14
5   0001-0525-0116-NucleoVerde  0.490  0.421      14
6   0001-0890-0989-NucleoVerde  0.449  0.426      16
7   0001-0280-0652-NucleoVerde  0.500  0.471      17
8   0001-0256-0702-NucleoVerde  0.469  0.464      17
9   0001-0210-0200-NucleoVerde  0.486  0.458      27
10  0001-0912-0350-NucleoVerde  0.411  0.435      27
----------------------------------------
Processing configuration: Toxin_vs_actin_ord
  Directory Path: /home/jovyan/LNMA/bravoa/data/New2 Fig para colocalizacion Manders-actina
  Order Types: ['-ord']
  Channel M1: 1
  Channel M2: 2
  Da

...

/home/jovyan/LNMA/bravoa/data/Seleccion ROIs circulares Lizo/Vesicles_Toxin_vs_Lizo_all_manders_results.csv
Results:                        roi     m1     m2  sample
0   0003-0512-0575-VE Lizo  0.469  0.487       1
1   0003-0345-0555-VE Lizo  0.479  0.448       1
2   0003-0767-0557-VE Lizo  0.524  0.509       1
3   0003-0024-0385-VE Lizo  0.499  0.497       1
4   0003-0261-0529-VE Lizo  0.475  0.500       1
5   0002-0274-0561-VE Lizo  0.505  0.508       2
6   0002-0765-0704-VE Lizo  0.589  0.621       2
7   0002-0529-0582-VE Lizo  0.481  0.477       2
8   0002-0661-0569-VE Lizo  0.470  0.498       2
9   0002-0645-0660-VE Lizo  0.533  0.505       2
10  0002-0797-0690-VE Lizo  0.583  0.558       3
11  0002-0291-0664-VE Lizo  0.572  0.541       3
12  0002-0879-0720-VE Lizo  0.453  0.486       3
13  0002-0234-0712-VE Lizo  0.469  0.475       3
14  0002-0513-0807-VE Lizo  0.365  0.489       4
15  0002-0603-0305-VE Lizo  0.236  0.470       4
16  0002-0924-0586-VE Lizo  0.506  0.535       5
1

progress: 100%|██████████| 9/9 [00:00<00:00, 1500.53it/s]


/home/jovyan/LNMA/bravoa/data/Seleccion ROIs circulares Mito/Vesicles_Toxin_vs_Mito_all_manders_results.csv
Results:                        roi     m1     m2  sample
0   0003-0703-0684-VE mito  0.537  0.335       1
1   0003-0519-0665-VE mito  0.566  0.491       1
2   0003-0499-0768-VE mito  0.489  0.485       1
3   0003-0471-0524-VE mito  0.495  0.454       1
4   0003-0267-0416-VE mito  0.537  0.433       1
..                     ...    ...    ...     ...
90  0002-0958-0347-VE mito  0.474  0.485      14
91  0002-0985-0450-VE mito  0.534  0.480      14
92  0002-0827-0381-VE mito  0.550  0.478      14
93  0002-0536-0143-VE mito  0.396  0.496      14
94  0002-0172-0087-VE mito  0.462  0.455      14

[95 rows x 4 columns]
----------------------------------------
Processing configuration: Vesicles_Toxin_vs_Endo
  Directory Path: /home/jovyan/LNMA/bravoa/data/Seleccion ROIs circulares Endosomas
  Order Types: ['endo']
  Channel M1: 1
  Channel M2: 3
  DataFrame Name: Vesicles_Toxin_vs_Endo
-

progress: 100%|██████████| 9/9 [00:00<00:00, 2560.28it/s]


/home/jovyan/LNMA/bravoa/data/Seleccion ROIs circulares Endosomas/Vesicles_Toxin_vs_Endo_all_manders_results.csv
Results:                         roi     m1     m2  sample
0    0002-0108-0168-VE endo  0.531  0.503       1
1    0002-0256-0050-VE endo  0.514  0.466       1
2    0002-0216-0048-VE endo  0.511  0.514       1
3    0002-0275-0316-VE endo  0.526  0.472       1
4    0002-0090-0243-VE endo  0.551  0.571       1
..                      ...    ...    ...     ...
106  0004-0877-0906-VE endo  0.511  0.501      14
107  0004-0403-0854-VE endo  0.512  0.500      14
108  0004-0221-0818-VE endo  0.502  0.468      14
109  0004-0339-0900-VE endo  0.502  0.447      14
110  0004-0989-0993-VE endo  0.482  0.472      14

[111 rows x 4 columns]
----------------------------------------
